In [17]:
import numpy as np

In [18]:
import os
import ctypes
import sys

# Clear any previous bad imports
modules_to_clean = []
for name in list(sys.modules.keys()):
    if any(substring in name for substring in ['h5py', 'prometheus', '_hdf5', '_h5py', '_errors']):
        modules_to_clean.append(name)

for mod in modules_to_clean:
    if mod in sys.modules:
        del sys.modules[mod]

# Set library path
intel_paths = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64",
]

current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')
new_ld_path = ':'.join(intel_paths + [current_ld_path])
os.environ['LD_LIBRARY_PATH'] = new_ld_path

# Manually preload Intel libraries BEFORE any imports
intel_libs = [
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5",
    "/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so",
]

# Load the libraries with RTLD_GLOBAL so they're available to subsequent imports
for lib_path in intel_libs:
    try:
        ctypes.CDLL(lib_path, mode=ctypes.RTLD_GLOBAL)
        print(f"Successfully preloaded: {lib_path}")
    except Exception as e:
        print(f"Failed to preload {lib_path}: {e}")

# Set Python paths
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus')
sys.path.insert(0, '/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

# Now try importing
try:
    import h5py
    print("✓ h5py imported successfully")
except Exception as e:
    print(f"✗ h5py import failed: {e}")

try:
    from prometheus import Prometheus, config
    print("✓ prometheus imported successfully!")
except Exception as e:
    print(f"✗ prometheus import failed: {e}")
    import traceback
    traceback.print_exc()

2025-05-07 00:51:33,488 - h5py._conv - DEBUG - Creating converter from 7 to 5
2025-05-07 00:51:33,489 - h5py._conv - DEBUG - Creating converter from 5 to 7
2025-05-07 00:51:33,490 - h5py._conv - DEBUG - Creating converter from 7 to 5
2025-05-07 00:51:33,490 - h5py._conv - DEBUG - Creating converter from 5 to 7


Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libimf.so
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libintlc.so.5
Successfully preloaded: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64/libsvml.so
✓ h5py imported successfully
✓ prometheus imported successfully!


Python executable: /groups/icecube/jackp/.conda/envs/myenv/bin/python
Python version: 3.9.19 (main, May  6 2024, 19:43:03) 
[GCC 11.2.0]

Conda environments:
# conda environments:
#
h5py_test_new            /groups/icecube/jackp/.conda/envs/h5py_test_new
myenv                 *  /groups/icecube/jackp/.conda/envs/myenv
myenv_310                /groups/icecube/jackp/.conda/envs/myenv_310
base                     /software/astro/anaconda/anaconda3-2020.11



Current conda environment: myenv

LD_LIBRARY_PATH: /software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64:/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64:/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/compiler/lib/intel64:/software/local/intel/parallel_studio_xe_2020_upd4/compilers_and_libraries_2020.4.304/linux/mkl/lib/intel64:/groups/icecube/jackp/geant4-v11.1.0-install/li

In [20]:
# import os

# # Set Intel library paths - adjust these based on your system
# intel_paths = [
#     "/opt/intel/compilers_and_libraries_2020.0.166/linux/compiler/lib/intel64",
#     "/opt/intel/compilers_and_libraries_2020.0.166/linux/mkl/lib/intel64",
#     # Add other Intel library paths as needed
# ]

# # Get current LD_LIBRARY_PATH
# current_ld_path = os.environ.get('LD_LIBRARY_PATH', '')

# # Add Intel paths
# new_ld_path = ':'.join(intel_paths + [current_ld_path])
# os.environ['LD_LIBRARY_PATH'] = new_ld_path

# # Now try importing
from prometheus import Prometheus, config

In [10]:
import os
os.chdir('/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples')

In [11]:
import pandas as pd
import numpy as np
import time
import sys
import argparse
import os
import logging
from datetime import datetime

sys.path.append('../')
from prometheus import Prometheus, config
import prometheus
import gc

# Import our utility functions
from genie_parser_injection import parse_and_convert_genie

from inject_in_cylinder import inject_particles_in_cylinder
from rotate_particles import rotate_particles_final

# Set up logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
RESOURCE_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/resources/"
OUTPUT_DIR = f"{'/'.join(prometheus.__path__[0].split('/')[:-1])}/examples/output"


In [12]:
import prometheus

In [15]:

def main():
    # Start timing
    start_time = time.time()

    # Use the arguments
    simset = 11
    root_file_path = '/groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root'
    
    print(f"Using simset: {simset}")
    print(f"Using root file: {root_file_path}")
    
    # Timing for file processing and conversion in one step
    processing_start_time = time.time()
    prometheus_set, primary_set = parse_and_convert_genie(root_file_path)
    processing_end_time = time.time()
    print(f"File processing and conversion completed in {processing_end_time - processing_start_time:.2f} seconds")

    # absolute path:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
   # output_dir = f"/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/examples/output"
    primrary_file_path = f'{OUTPUT_DIR}/genie_events_primary_new_test.parquet'
    prometheus_file_path = f'{OUTPUT_DIR}/genie_events_prometheus_new_test.parquet'

    # inject into cylinder:
    primary_set, prometheus_set = inject_particles_in_cylinder(primary_set, ## neutrino information
                        prometheus_set, ## child particle information
                        cylinder_radius=500, ## meters
                        cylinder_height = 1000, # meters
                        cylinder_center = (0, 0, 0), # meters not including offset
                        )

    primary_set, prometheus_set = rotate_particles_final(primary_set, prometheus_set)


    # Timing for serialization and saving
    save_start_time = time.time()
    # position arrays to serialized strings
    prometheus_set['position'] = prometheus_set['position'].apply(lambda x: [arr.tolist() for arr in x])
    num_events = len(prometheus_set)
    print(f"Processing {num_events} events")
    prometheus_set.to_parquet(prometheus_file_path)
    primary_set.to_parquet(primrary_file_path)
    save_end_time = time.time()
    print(f"Serialization and saving completed in {save_end_time - save_start_time:.2f} seconds")

    ## GENIE stuff:
    config["injection"]["name"] = "GENIE"
    config["run"]["outfile"] = f"{OUTPUT_DIR}/100_new_lots_events_{timestamp}.parquet"
    config["run"]["nevents"] = num_events
    config["injection"]["GENIE"] = config["injection"].get("GENIE", {})
    config["injection"]["GENIE"]["paths"] = config["injection"]["GENIE"].get("paths", {})
    config["injection"]["GENIE"]["inject"] = False ## we aren't injecting using prometheus, we are using an injection file
    config["injection"]["GENIE"]["simulation"] = {}
    ## geofile:
    config["detector"]["geo file"] = f"{RESOURCE_DIR}/geofiles/icecube.geo"
    ## ppc configuration:
    config['photon propagator']['name'] = 'PPC'
    config["photon propagator"]["PPC"] = config["photon propagator"].get("PPC", {})
    config["photon propagator"]["PPC"]["paths"] = config["photon propagator"]["PPC"].get("paths", {})
    config["photon propagator"]["PPC"]["paths"]["ppc_tmpdir"] = "./ppc_tmpdir"
    config["photon propagator"]["PPC"]["paths"]["ppctables"] = f"{RESOURCE_DIR}/PPC_tables/spx" ## spx for test
    
    config["photon propagator"]["PPC"]["paths"]["ppc_tmpfile"] = "ppc_tmp"+str(simset)
    config["photon propagator"]["PPC"]["simulation"]["supress_output"] = False ## for printing!

    # Timing for Prometheus simulation
    sim_start_time = time.time()
    p = Prometheus(config, primary_set_parquet_path=primrary_file_path, prometheus_set_parquet_path=prometheus_file_path)
    p.sim()
    sim_end_time = time.time()
    print(f"Prometheus simulation completed in {sim_end_time - sim_start_time:.2f} seconds")
    
    # End timing
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print('Finished without catastrophic error')
    return



In [16]:


print("--------------------------------------------------------------")
print("--------------------------------------------------------------")
print("Launching simulation")
main()
print("Finished call")
print("--------------------------------------------------------------")
print("--------------------------------------------------------------")

2025-05-06 22:34:18,690 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-06 22:34:18,691 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-06 22:34:18,693 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-06 22:34:18,696 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-06 22:34:18,710 - asyncio - DEBUG - Using selector: EpollSelector
2025-05-06 22:34:18,711 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-06 22:34:18,721 - fsspec.local - DEBUG - open file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
2025-05-06 22:34:18,722 - fsspec.local - DEBUG - o

--------------------------------------------------------------
--------------------------------------------------------------
Launching simulation
Using simset: 11
Using root file: /groups/icecube/jackp/genie_test_outputs/output_gheps/gntp_icecube_numu_100.gtac.root
File processing and conversion completed in 0.11 seconds


/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/genie_examples/genie_parser_injection.py:20: RuntimeWarning: invalid value encountered in scalar divide
  angle = np.arccos(np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1))


Event 0, Particle 0:
  Original: theta=1.6347, phi=3.0832
  Rotated: theta=1.8373, phi=-2.8265
  Changed: True
Event 0, Particle 1:
  Original: theta=0.0760, phi=0.2222
  Rotated: theta=1.6821, phi=1.7122
  Changed: True

After assignment:
  First theta original: 0.0491
  First theta in DataFrame: 1.8373
  Changed in DataFrame: True
Event 1, Particle 0:
  Original: theta=1.5713, phi=2.4640
  Rotated: theta=2.1313, phi=2.3874
  Changed: True
Event 1, Particle 1:
  Original: theta=0.1273, phi=0.9933
  Rotated: theta=0.5874, phi=2.6383
  Changed: True
Processing 100 events
Serialization and saving completed in 0.08 seconds
using injector {'name': 'GENIE', 'GENIE': {'inject': False, 'paths': {'injection file': None}}}
position: [ 266.16086809 -376.63526167 -305.66703702]
detector offset [    5.87082946    -2.51860853 -1971.9757655 ]
relative_position,  [  272.03169755  -379.1538702  -2277.64280252]
position: [-400.79410834   18.42091668  192.25263102]
detector offset [    5.87082946    -2.

Propagating MuMinus:   0%|          | 0/100 [00:00<?, ?it/s]/groups/icecube/jackp/prometheus_genie_cleaned/harvard-prometheus/prometheus/lepton_propagation/new_proposal_lepton_propagator.py:197: UserWarning: Adjusting ecut and vcut to work with ppc
  warnings.warn("Adjusting ecut and vcut to work with ppc", UserWarning)


...

returning hadron 2112
returning hadron 2112


Error: too many random multipliers (150000), truncating to 131072
Loaded 131072 random multipliers
Loaded 42 wavelenth points
Loaded 171 ice layers
Loaded 5160 DOMs (21x19)

Processing f2k muons from stdin on device 0
photons: 9534  hits: 0
Propagating Neutron:  99%|█████████▉| 99/100 [01:44<00:01,  1.17s/it]2025-05-06 22:36:06,675 - prometheus.utils.write_to_f2k - DEBUG - TR 0 0 hadr -81.84466886769403 139.7425919875066 97.31301111151106 1.4766106449366372 2.541637302824945 0 1.5418045230967257 0.0
Configuring in "./ppc_tmpdir/"
Configured: xR=1 eff=0.9 sf=0 g=0.8
Loaded 12 angsens coefficients
Loaded 6x170 dust layer points
Error: too many random multipliers (150000), truncating to 131072
Loaded 131072 random multipliers
Loaded 42 wavelenth points
Loaded 171 ice layers
Loaded 5160 DOMs (21x19)

Processing f2k muons from stdin on device 0
photons: 1377  hits: 0
Propagating PiPlus:  99%|█████████▉| 99/100 [01:44<00:01,  1.17s/it] 2025-05-06 22:36:06,808 - prometheus.utils.write_to_f2k 

returning hadron 2112
Handling charged pion/kaon 211


Configuring in "./ppc_tmpdir/"
Configured: xR=1 eff=0.9 sf=0 g=0.8
Loaded 12 angsens coefficients
Loaded 6x170 dust layer points
Error: too many random multipliers (150000), truncating to 131072
Loaded 131072 random multipliers
Loaded 42 wavelenth points
Loaded 171 ice layers
Loaded 5160 DOMs (21x19)

Processing f2k muons from stdin on device 0
photons: 1448  hits: 0
Propagating PiPlus: 100%|██████████| 100/100 [01:44<00:00,  1.05s/it]


Prometheus simulation completed in 108.02 seconds
Total execution time: 108.35 seconds (1.81 minutes)
Finished without catastrophic error
I am melting.... AHHHHHH!!!!
Finished call
--------------------------------------------------------------
--------------------------------------------------------------
